# Описание проекта

Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.

# Задача проекта 

Разработать решение, позволяющее персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.

# Путь решения задачи

Нужно построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца.
В исследование нужно включить дополнительные данные финансового департамента о прибыльности клиента: какой доход каждый покупатель приносил компании за последние три месяца.
Используя данные модели и данные о прибыльности клиентов, нужно выделить сегменты покупателей и разработать для них персонализированные предложения.

# Описание данных

__market_file.csv__ - Таблица, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.

- `id` — номер покупателя в корпоративной базе данных.
- `Покупательская активность` — рассчитанный класс покупательской активности (целевой признак): «снизилась» или «прежний уровень».
- `Тип сервиса` — уровень сервиса, например «премиум» и «стандарт».
- `Разрешить сообщать` — информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель.
- `Маркет_актив_6_мес` — среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента.
- `Маркет_актив_тек_мес` — количество маркетинговых коммуникаций в текущем месяце.
- `Длительность` — значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте.
- `Акционные_покупки` — среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев.
- `Популярная_категория` — самая популярная категория товаров у покупателя за последние 6 месяцев.
- `Средний_просмотр_категорий_за_визит` — показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца.
- `Неоплаченные_продукты_штук_квартал` — общее число неоплаченных товаров в корзине за последние 3 месяца.
- `Ошибка_сервиса` — число сбоев, которые коснулись покупателя во время посещения сайта.
- `Страниц_за_визит` — среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.


__market_money.csv__ - Таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.

- `id` — номер покупателя в корпоративной базе данных.
- `Период` — название периода, во время которого зафиксирована выручка. Например, 'текущий_месяц' или 'предыдущий_месяц'.
- `Выручка` — сумма выручки за период.


__market_time.csv__ - Таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.

- `id` — номер покупателя в корпоративной базе данных.
- `Период` — название периода, во время которого зафиксировано общее время.
- `минут` — значение времени, проведённого на сайте, в минутах.


__money.csv__ - Таблица с данными о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.

- `id` — номер покупателя в корпоративной базе данных.
- `Прибыль` — значение прибыли.

In [ ]:
# импорт библиотек и инструментов
!pip install phik -q
!pip install shap -q
!pip install --upgrade scikit-learn
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import (
    StandardScaler, 
    OneHotEncoder, 
    OrdinalEncoder, 
    MinMaxScaler, 
    LabelEncoder
)
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    roc_auc_score,
    f1_score
)
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer 
from sklearn.feature_selection import SelectKBest, f_classif
from phik import phik_matrix
from phik.report import plot_correlation_matrix
import shap
import matplotlib.cm as cm

## Загрузка данных

In [ ]:
# загрузка данных
try:
# данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.
    df_market_file = pd.read_csv('/datasets/market_file.csv') 
# данные о выручке, которую получает магазин с покупателя  
    df_market_money = pd.read_csv('/datasets/market_money.csv') 
# данные о времени (в минутах), которое покупатель провёл на сайте в течение периода.
    df_market_time = pd.read_csv('/datasets/market_time.csv')
# данные о среднемесячной прибыли покупателя за последние 3 месяца
    df_money = pd.read_csv('/datasets/money.csv', sep=';', decimal=',')
except:
    df_market_file= pd.read_csv('https://code.s3.yandex.net/datasets/market_file.csv') 
    df_market_money = pd.read_csv('https://code.s3.yandex.net/datasets/market_money.csv') 
    df_market_time = pd.read_csv('https://code.s3.yandex.net/datasets/market_time.csv') 
    df_money = pd.read_csv('https://code.s3.yandex.net/datasets/money.csv', sep=';', decimal=',') 

In [ ]:
# переменная со списком используемых датафреймов
df_names = [df_market_file, df_market_money, df_market_time, df_money]

In [ ]:
def df_summary(df):
    """
    Отображает:
    - первые 5 строк датафрейма;
    - общую информацию о датафрейме;
    - количество и долю пропусков в датафрейме;
    - количество явных дубликатов;
    - уникальные значения для категориальных столбцов.

    Параметры:
    - df (pd.DataFrame): исследуемый датафрейм
    
    """
    print('========BEGIN========\n')
    print("\nПервые 5 строк данных:")
    display(df.head())
    
    print("\nОбщая информация о данных:")
    df.info()
    
    # проверка пропусков
    print("\nПропуски в данных:")
    missing_values = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().mean() * 100).sort_values(ascending=False)
    missing_info = pd.DataFrame({
        "Пропуски (шт.)": missing_values,
        "Пропуски (%)": missing_percent
    })
    display(missing_info)
    
    # проверка явных дубликатов
    duplicates = df.duplicated().sum()
    print(f"\nКоличество явных дубликатов: {duplicates}")
    
    # уникальные значения для категориальных столбцов
    print("\nУникальные значения для категориальных столбцов:")
    for column in df.select_dtypes(include=["object"]).columns:
        print(f"\nСтолбец: {column}")
        display(df[column].value_counts())
    print('\n========END========')

In [ ]:
# отображение общей информации о датафреймах
for df in df_names:
    df_summary(df)

__Промежуточный вывод по загрузке данных:__
 - данные в таблицах соответствуют описанию;
 - наименования столбцов в таблицах не соответствуют стандарту pep8 - далее приведем регистр наименований к нижнему и заменим пробелы символом `_`;
 - в таблицах отсутствуют пропуски и явные дубликаты;
 - в датафрейме `df_market_file` в столбце `Тип сервиса` имеется неявный дубликат - опечатка в значении _стандартт_, а также опечатка _Косметика и аксесуары_ - далее устраним дублирование и опечатку.
 - в датафрейме `df_market_time` в столбце `Период` имеется - опечатка в значении _предыдцщий_месяц_ - далее устраним опечатку.
 


## Предобрабокта данных

In [ ]:
# приведение названий столбцов к нижнему регистру и замена пробелов на _
for df in df_names:
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    print(df.columns)

In [ ]:
# удаляем неявный дубликат в столбце тип_сервиса
df_market_file['тип_сервиса'] = df_market_file['тип_сервиса'].str.replace('стандартт', 'стандарт')
# исправляем опечатку в столбце популярная категория
df_market_file['популярная_категория'] = df_market_file['популярная_категория'] \
    .str.replace('Косметика и аксесуары', 'Косметика и аксессуары')

print(df_market_file['тип_сервиса'].value_counts())
print(df_market_file['популярная_категория'].value_counts())

# исправляем опечатку в столбце период
df_market_time['период'] = df_market_time['период'].str.replace('предыдцщий_месяц', 'предыдущий_месяц')
print(df_market_time['период'].value_counts())

__Промежуточный вывод по предобработке данных:__
 - название столбцов в датафреймах приведены к нижнему регистру, пробелы заменены на символ `_`;
 - удалены неявные дубликаты, обнаруженные на шаге загрузки данных;
 

## Исследовательский анализ

In [ ]:
# определим функцию для графического отображения данных в датафреймах
def plot_stat_data_analysis(df):
    for column in df.columns:
        # исключаем обработку столбца с уникальным номер
        if column != 'id':
            # определяем размер отображения графиков
            plt.figure(figsize=(6, 4))
            if df[column].dtype in ['int64', 'float64']:
                # для количественных данных строим гистограмму
                df[column].hist(bins=30, 
                                color='#0ff', 
                                edgecolor='black')
                # отображение среднего значения
                plt.axvline(df[column].mean(),
                            color='red',
                            linestyle='--',
                            label="Среднее значение")
                # отображение медианного значения
                plt.axvline(df[column].median(),
                            color='red',
                            linestyle='-',
                            label="Медиана")            
                plt.title(f'Гистограмма \"{column}\"')
                plt.xlabel(column)
                plt.ylabel('Частота')
                plt.legend()
            elif df[column].dtype == 'object':
                # для категориальных данных строим круговую диаграмму
                df[column].value_counts().plot(kind='pie',
                                           autopct='%1.1f%%', 
                                           startangle=90, 
                                           wedgeprops={'edgecolor': 'black'})
                plt.title(f'Круговая диаграмма \"{column}\"')
                plt.xlabel('')
                plt.ylabel('')
            else:
                # Пропускаем столбцы с неподдерживаемыми типами
                plt.close()
                continue
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

In [ ]:
for i, df in enumerate(df_names, start=1):
    print(f"Графическое отображение данных датафрейма №{i}")
    plot_stat_data_analysis(df)

In [ ]:
# вывод столбчатых диаграмм для признаков с дискретными значениями
for column in ['маркет_актив_тек_мес',
               'средний_просмотр_категорий_за_визит',
               'неоплаченные_продукты_штук_квартал',
               'ошибка_сервиса']:
    plt.figure(figsize=(6, 4))
    value_counts = df_market_file[column].value_counts()
    colors = cm.Blues(value_counts / max(value_counts))
    value_counts.plot(kind='barh', 
                      color=colors,
                      edgecolor='black')
    plt.title(f'Столбчатая диаграмма \"{column}\"')
    plt.xlabel('Количество пользователей')
    plt.ylabel('Значения')

In [ ]:
# вывод описательной статистики по исходным датафреймам
for i, df in enumerate(df_names, start=1):
    print(f"Графическое отображение данных датафрейма №{i}")
    display(df.describe())

__Промежуточный вывод__:
В целом, данные представлены без аномалий, за исключением признака `выручка` в датафрейме `df_market_money`, требуется уточнение.


In [ ]:
# оценка выбросов через диаграмму размаха для столбца выручка
df_market_money['выручка'].plot.box()
plt.title('Диаграмма размаха для Выручки')
plt.ylabel('Выручка')
plt.show()

Присутствует один существенный выброс. Удалим покупателя с аномальной выручкой из всех датафреймов.

In [ ]:
# удаление выброса с аномальным значением выручки
max_revenue = df_market_money[df_market_money['выручка'] == df_market_money['выручка'].max()]['id'].tolist()
df_market_file = df_market_file.query('id not in @max_revenue')
df_market_money = df_market_money.query('id not in @max_revenue')
df_market_time = df_market_time.query('id not in @max_revenue')
df_money = df_money.query('id not in @max_revenue')

In [ ]:
# построение граифка распределения значений в столбце выручка после удаления аномального 
df_market_money['выручка'].hist(bins=20,
                           color='skyblue', 
                           edgecolor='black')
plt.axvline(df_market_money['выручка'].mean(),
            color='red',
            linestyle='--',
            label="Среднее значение")
plt.axvline(df_market_money['выручка'].median(),
            color='red',
            linestyle='-',
            label="Медиана")          
plt.title(f'Гистограмма Выручка')
plt.ylabel('Частота')
plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0,)
plt.show()

Значения выручки распределены нормально, но имеются выбросы около нулевого значения. Необходимо оценить их количество.

In [ ]:
# оценка количества покупателей, не принесших выручку в течение последних 3 месяцов
df_market_money.query('выручка == 0')['id'].unique()

Эти пользователи не проявляли активность более трех месяцев, их необходимо будет удалить из всех датафреймов.

In [ ]:
# выбор неактивных пользователей
inactive_customers = df_market_money.query('выручка == 0')['id'].tolist()

In [ ]:
df_market_file = df_market_file.query('id not in @inactive_customers')
df_market_money = df_market_money.query('id not in @inactive_customers')
df_market_time = df_market_time.query('id not in @inactive_customers')
df_money = df_money.query('id not in @inactive_customers')

In [ ]:
# построение граифка распределения значений в столбце выручка после удаления пользователей с нулевой выручкой
df_market_money['выручка'].hist(bins=20,
                           color='skyblue', 
                           edgecolor='black')
plt.axvline(df_market_money['выручка'].mean(),
            color='red',
            linestyle='--',
            label="Среднее значение")
plt.axvline(df_market_money['выручка'].median(),
            color='red',
            linestyle='-',
            label="Медиана")          
plt.title(f'Гистограмма Выручка')
plt.ylabel('Частота')
plt.legend(bbox_to_anchor=(1.04, 1), borderaxespad=0,)
plt.show()
df_market_money['выручка'].describe()

In [ ]:
# перезапись списка с исходными датафреймами после преобразования
df_names = [df_market_file, df_market_money, df_market_time, df_money]
# вывод на экран сведений об обновленных датафреймах
for df in df_names:
    print(f'{df.info()}\n')

__Промежуточный вывод по исследовательскому анализу:__

__Датафрейм `df_market_file`__ - таблица, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.
- признак `покупательская_активность` (целевой) - представлен в виде двух групп: 38.3% - снизилась, 61.7% - прежний уровень. Бинарный, категориальный. Имеется небольшой дисбаланс. При разделении на тренировочную и тестовую выборку используем стратификацию. 
- признак `тип_сервиса` - представлен в виде двух групп: премиум - 28.9%, стандарт - 71.1%. Бинарный, категориальный. Большинство покупателей предпочитают стандартный уровень сервиса. 
- признак `разрешить_сообщать` - представлен в виде двух групп: да - 74%, нет - 26%. Бинарный, категориальный. Большинство покупателей предпочитают не получать дополнительные предложения о товарах. 
- признак `маркет_актив_6_мес` - количественный признак с распределением близким к нормальному преимущественно в диапазоне от 1 до 6 активностей.
- признак `маркет_актив_тек_мес` - количественный признак, представленный в виде 3х значений: доминирующее количество активностей - 4 (около 50% значений), на 3 и 5 активностей приходится примерно по 25% значений.
- признак `длительность` - количественный признак с нормальным распределением.
- признак `акционные_покупки` - количественный признак с бимодальным распределением. Присутствуют 2 характерных кластера: 
  - первый, больший, сосредоточен в районе доли 0.2;
  - второй сосредоточен около доли 1 - покупатели, которые покупают товар преимущественно по акции.
- признак `популярная_категория` - категориальный признак, представленный в виде 6 групп:
  - Кухонная посуда - 10.6%;
  - Мелкая бытовая техника и электроника - 13.4%;
  - Техника для красоты и здоровья - 14.2%;
  - Косметика и аксессуары - 17.2%;
  - Домашний текстиль - 19.3%;
  - Товары для детей - 25.4%.
- признак `средний_просмотр_категорий_за_визит` - количественный признак с немного смещенным распределением. Всего представлено 6 значений (соответствует максимальному числу категорий товаров). Медианное значение - 3, среднее значение - 3.27.
- признак `неоплаченные_продукты_штук_квартал` - количественный признак с распределением, смещенным влево. Минимальное значение - 0, максимальное значение - 10. Среднее значение - 2.84, медианное значение - 3. 
- признак `ошибка_сервиса` - количественный признак с нормальным распределением. Минимальное значение - 0, максимальное - 9.
- признак `страниц_за_визит` - количественный признак с нормальным распределением. Минимальное значение - 1, максимальное - 20.
Аномальных значений в представленных признаках отсутствуют.

__Датафрейм `df_market_money`__ - таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.
 - признак `период` - категориальный признак, представленный в виде 3х равномерно распределенных значений - текущий_месяц (33%), предыдущий_месяц (33%), препредыдущий_месяц (33%).
 - признак `выручка` - количественный признак с нормальным распределением.

Присутсвовали аномальные значения и выбросы в признаке `выручка`. 

__Датафрейм `df_market_time`__ - таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.
 - признак `период` - представлен в виде 2х групп: текущий месяц - 50%, предыдущий месяц - 50%. Бинарный, категориальный.
 - признак `минут` - количественный признак с нормальным распределением. Минимальное значение - 4, максимальное значение - 23.
Аномальных значений в представленных признаках отсутствуют.

__Датафрейм `df_money`__ - таблица с данными о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.
 - признак `прибыль` -  количественный признак с нормальным распределением. Аномальные значения отсутствуют. 
 
По результаты анализа датафреймов из них удалены строки соответствующие `id` покупателей, которые не проявляли активность за последние 3 месяца (ничего не покупали), в количестве 3х штук и `id` покупателя, выручка по которому была аномальной. 

Кроме того, в дальнейшем, некоторые признаки можно попробовать преобразовать из количественных в категориальные, например:
 - признак `акционные_покупки`, разделив покупателей на 2 кластера: с долей акционных покупок менее 0.5 и не менее 0.5. 
 - признаки `средний_просмотр_категорий_за_визит`, `неоплаченные_продукты_штук_квартал`, `ошибка_сервиса`, учитывая дискретность величин и небольшое количество уникальных значений.

## Объединение таблиц

In [ ]:
# преобразование df_market_money в широкую таблицу
df_market_money_pivot = df_market_money.pivot_table(index='id', 
                                                    columns='период', 
                                                    values='выручка')
# переименование столбцов для удобства идентификации выручки за период
df_market_money_pivot.columns = [f'выручка_{col}' for col in df_market_money_pivot.columns]

# преобразование df_market_time в широкую таблицу
df_market_time_pivot = df_market_time.pivot_table(index= 'id', 
                                                  columns='период', 
                                                  values='минут')
# переименование столбцов для удобства идентификации времени за период
df_market_time_pivot.columns = [f'минут_{col}' for col in df_market_time_pivot.columns]

In [ ]:
# объединение таблиц df_market_file и широких вариаций df_market_money и df_market_time
df_market_file = df_market_file.merge(df_market_money_pivot,
                                     on='id',
                                     how='left').merge(df_market_time_pivot,
                                                      on='id',
                                                     how='left')

In [ ]:
# вывод сведений об объединенном датафрейме
display(df_market_file.head())
display(df_market_file.info())

__Промежуточный вывод по объединению таблиц__:

Данные успешно объединены в единый датафрейм.

### Уточнение исследовательского анализа после объединения таблиц

In [ ]:
# построение распределений количественных признаков с учетом покупательской активности
for column in df_market_file.select_dtypes(include=['number']).columns[1:]:
    plt.figure(figsize=(13,6))
    plt.subplot(1, 2, 1)
    plt.title(f'Гистограмма для "{column}"')
    sns.histplot(data=df_market_file,
                 x=column,
                 hue='покупательская_активность',
                 palette='YlGn')
    plt.xlabel(f'Признак "{column}"')
    plt.ylabel('Частота')
    plt.tight_layout()

In [ ]:
# построение круговых диаграмм по категориальным признакам с учетом покупательской активности
for column in df_market_file.select_dtypes(include=['object']).columns[1:]:
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))  

    # первая диаграмма по фильтру покупательская_активность - снизилась
    df_market_file[df_market_file['покупательская_активность'] == 'Снизилась'][column].value_counts().plot(
        kind='pie',
        autopct='%1.1f%%',
        startangle=90,
        wedgeprops={'edgecolor': 'black'},
        ax=axes[0]
    )
    axes[0].set_title(f'Круговая диаграмма \"{column}\" (Снизилась)')
    axes[0].set_xlabel('')
    axes[0].set_ylabel('')

    # вторая диаграмма по фильтру покупательская_активность - прежний уровень
    df_market_file[df_market_file['покупательская_активность'] == 'Прежний уровень'][column].value_counts().plot(
        kind='pie',
        autopct='%1.1f%%',
        startangle=90,
        wedgeprops={'edgecolor': 'black'},
        ax=axes[1]
    )
    axes[1].set_title(f'Круговая диаграмма \"{column}\" (Прежний уровень)')
    axes[1].set_xlabel('')
    axes[1].set_ylabel('')

    plt.tight_layout() 
    plt.show()

__Промежуточный вывод по исследовательскому анализу признаков с учетом целекого__:

Результат сопоставление признаков двух групп покупателей: со сниженной покупательской активностью и покупательской активностью на прежнем уровне.
- признак `маркет_актив_6_мес` - для покупателей, чей уровень активности не изменился, частота маркетинговой активности за последние 6 месяцев выше.
- признак `маркет_актив_тек_мес` - распределения частоты маркетинговой активности для покупателей обеих групп идентичны.
- признак `длительность` - покупатели с прежним уровнем активности в среднем являются пользователями сервиса чуть меньше, чем покупатели со сниженным уровнем.
- признак `акционные_покупки` - пользователи, чья покупательская активность снизилась, за последние 6 месяцев чаще покупали товары по акции.
- признак `средний_просмотр_категорий_за_визит` - пользователи, чья покупательская активность снизилась, просматривают меньшее количество категорий.
- признак `неоплаченные_продукты_штук_квартал` - пользователи, чья покупательская активность не изменилась, имеют в корзине меньшее количество неоплаченных товаров.
- признак `ошибка_сервиса` - покупатели, чей уровень активности не изменился (пока), чаще сталкиваются со сбоями при посещении сайта.
- признак `страниц_за_визит`- покупатели, чья активность снизилась, просматривают сильно меньше страниц за одно посещение.
- признак `выручка_предыдущий_месяц` - распределения для обеих групп идентичны.
- признак `выручка_препредыдущий_месяц` - пользователи, чей уровень активности снизился, 2 месяца назад принесли меньше выручки.
- признак `выручка_текущий_месяц` - распределение у группы с прежним уровнем активности имеет форму более вытянутого конуса, что говорит о более предсказуемом объеме выручки.
- признак `минут_предыдущий_месяц` - пользователи, чья активность снизилась, меньше проводили времени на сайте.
- признак `минут_текущий_месяц` - пользователи, чья активность снизилась, меньше проводят времени на сайте.
- признак `тип_сервиса` - доля пользователей с типом сервиса _премиум_ в группе, чья активность снизилась, чуть выше, чем в группе с прежним уровнем.
- признак `разрешить_сообщать` - в обеих группа, доля пользователей, кто разрешил рассылку в свой адрес, одинаковая.
- признак `популярная категория` - в обеих группах категория товаров для детей занимает первое место. Рейтинг категория товаров для пользователей с разной покупательской активностью:
 - Снизилась:
   - Товары для детей;
   - Косметика и аксессуары;
   - Домашний текстиль;
   - Кухонная посуда;
   - Техника для красоты и здоровья;
   - Мелкая бытовая техника и электроника;
 - Прежний уровень:
   - Товары для детей;
   - Домашний текстиль;
   - Мелкая бытовая техника и электроника;
   - Техника для красоты и здоровья;
   - Косметика и аксессуары;
   - Кухонная посуда.

В общем, пользователи, чья покупательская активность снизилась, вероятнее используют сервис для целенаправленной покупки товаров по акции, т.к. в среднем делают это чаще, категорий перебирают меньше и меньше тратят времени на просмотр страниц. У таких пользователей больше неоплаченных товаров в корзине, что может свидетельствовать о том, что происходит выжидание какой-либо акции. Также можно предположить, что 2 месяца назад акции было меньше, с чем связано падение выручки по этой группе пользователей. Кроме того, такие пользователи меньше получали рассылок и предложений. 

p.s. При обучении моделей признаки `маркет_актив_тек_мес`, `выручка_предыдущий_месяц`, `разрешить_сообщать` и `тип_сервиса`, вероятно, будут оказывать меньшее влияние на предсказание целевого, т.к. идентичны для обеих групп.

## Корреляционный анализ

In [ ]:
# установим в качестве индекса id пользователя
df_market_file = df_market_file.set_index('id')

In [ ]:
# расчет корреляции
corr_matrix = df_market_file.phik_matrix(interval_cols=['маркет_актив_6_мес', 
                                                        #'маркет_актив_тек_мес', - дискретный, мало значений
                                                        'длительность', 
                                                        'акционные_покупки', 
                                                        #'средний_просмотр_категорий_за_визит', - дискретный, мало значений
                                                        #'неоплаченные_продукты_штук_квартал', - дискретный, мало значений
                                                        #'ошибка_сервиса', - дискретный, мало значений
                                                        #'страниц_за_визит', - дискретный, мало значений
                                                        'выручка_предыдущий_месяц', 
                                                        'выручка_препредыдущий_месяц', 
                                                        'выручка_текущий_месяц', 
                                                        'минут_предыдущий_месяц', 
                                                        'минут_текущий_месяц'])
# визуализация матрицы корреляции
plot_correlation_matrix(
    corr_matrix.values,
    x_labels=corr_matrix.columns,
    y_labels=corr_matrix.index,
    vmin=0, vmax=1, color_map='Greens',
    title='Матрица корреляции',
    fontsize_factor=1.5,
    figsize=(20, 15)
) 

__Промежуточный вывод по корреляционному анализу__:

С целевым признаком `покупательская_активность` наблюдается некоторая корреляция со следующими признаками:
 - `маркет_актив_6_мес`, коэф 0.54;
 - `акционные_покупки`, коэф 0.51;
 - `средний_просмотр_категорий_за_визит`, коэф 0.54;
 - `неоплаченные_продукты_штук_квартал`, коэф 0.51;
 - `страниц_за_визит`, коэф 0.75 - _самый высокий_;
 - `выручка_препредыдущий_месяц`, коэф 0.50;
 - `минут_предыдущий_месяц`, коэф 0.69;
 - `минут_текущий_месяц`, коэф 0.58


Среди входных признаков наблюдается сильная корреляция между `выручка_предыдущий_месяц` и `выручка_текущий_месяц`. Для устранения мультиколлинеарности перед подбором модели можно объединить эти признаки в единый `выручка_последние_2_месяца`.

## Использование пайплайнов

На основании исследовательского и корреляционного анализов преобразуем некоторые признаки:
- признак `акционные_покупки` преобразуем в категориальный `акционные_покупки_объем`: _много покупают по акции_ при доле акционных покупок не менее 0.5 и _мало покупают по акции_ при доле покупок менее 0.5;
- признаки `выручка_предыдущий_месяц` и `выручка_текущий_месяц` путем суммирования заменим одним `выручка_последние_2_месяца`.

При подготовке тренировочной и тестовой выборок лишние признаки удалим.

In [ ]:
# преобразование и добавление признаков

df_market_file['акционные_покупки_объем'] = df_market_file['акционные_покупки']\
.apply(lambda x: 'много покупают по акции' if x>= 0.5 else 'мало покупают по акции')

df_market_file['выручка_последние_2_месяца'] = df_market_file['выручка_предыдущий_месяц']\
+ df_market_file['выручка_текущий_месяц']

In [ ]:
# вывод преобразованного датафрейма
df_market_file.head()

In [ ]:
df_market_file.info()

In [ ]:
# определение констант
TEST_SIZE = 0.25
RANDOM_STATE = 42

Учитывая формы распределений количественных признаков, в качестве скейлеров будем пробовать использовать `StandardScaler()` и `MinMaxScaler()`, а также опцию `passthrough` для гибкости.

Целевой признак закодируем с помощью `LabelEncoder`.

Категориальные входные признаки закодируем с помощью `OneHotEncoder`, кроме условно ранжируемого `акционные_покупки_объем` - для него используем `OrdinalEncoder()`.

Обучать будем 4 модели: `KNeighborsClassifier()`, `DecisionTreeClassifier()`, `LogisticRegression()`, `SVC()`.
При назначении диапазонов гиперпараметров учтем:
- объем данных - небольшой (до 10 000 строк и до 100 признаков);
- коэффициенты корреляции.

Перебор будем использовать сплошной `GridSearchCV()`. 

Иные пояснения представлены в комментарях к коду.


In [ ]:
# разбиение данных на выборки
# исключение неиспользуемых признаков 
X = df_market_file.drop(['покупательская_активность',
                         'акционные_покупки',
                         'выручка_предыдущий_месяц',
                         'выручка_текущий_месяц'], axis=1)
y = df_market_file['покупательская_активность']
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = y)

In [ ]:
# кодирование целевого признака
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [ ]:
# определение признаков для предобработки 
ohe_columns = ['тип_сервиса',
               'разрешить_сообщать',
               'популярная_категория']

ord_columns = ['акционные_покупки_объем']

num_columns = ['маркет_актив_6_мес', 
               'маркет_актив_тек_мес', 
               'длительность',
               'средний_просмотр_категорий_за_визит',
               'неоплаченные_продукты_штук_квартал',
               'ошибка_сервиса',
               'страниц_за_визит',
               'выручка_препредыдущий_месяц',
               'минут_предыдущий_месяц',
               'минут_текущий_месяц',
               'выручка_последние_2_месяца']

In [ ]:
# пайплайн для кодирования категориальных признаков
ohe_pipe = Pipeline(
    [
        ('simpleImputer_ohe', SimpleImputer(missing_values=np.nan,
                                           strategy='most_frequent')),
        ('ohe', OneHotEncoder(drop='first', # сокращение объема признаков
                             handle_unknown='ignore',
                             sparse_output=False))
    ]
)

In [ ]:
# пайплайн для кодирования категориальных условно ранжированных признаков
ord_pipe = Pipeline(
    [
        (
            'simple_imputer_ord_before',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',
            OrdinalEncoder(categories=[
                                      ['мало покупают по акции','много покупают по акции']],
                          handle_unknown='use_encoded_value',
                          unknown_value=np.nan)
        ),
        (
            'simple_imputer_ord_after',
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

In [ ]:
# общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('ord', ord_pipe, ord_columns),
        ('num', 'passthrough', num_columns) # заглуша для задания динамического масштабирования
    ], 
    remainder='passthrough'
) 

In [ ]:
# итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', 'passthrough')  # заглушка для модели
])

In [ ]:
param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 6), # ограничение диапозона для предотвращения переобучения
        'models__max_features': range(2,6), # ограничение диапозона для предотвращения переобучения
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    
    # словарь для модели KNeighborsClassifier() 
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2,10), # проход от малых до средних значений
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']   
    },

    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear', #  оптимален для небольших данных и подходит для бинарной классификации
            penalty='l1' # совместим с solver='liblinear'
        )],
        'models__C': np.logspace(-1, 1, 10), # перебор оптимальных значений
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
    },
    # словарь для модели SVC()
    {
        'models': [SVC(random_state=RANDOM_STATE, probability=True)],

        # для разных типов ядер настраиваем соответствующие параметры

        # Полиномиальное ядро
        'models__kernel': ['poly'], # для обнуружения сложных полиноминальных зависимостей
        'models__degree': [2, 3],
        'models__C': [0.1, 1, 10],
        'models__gamma': ['scale', 'auto'],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough'],

        # RBF-ядро
        'models__kernel': ['rbf'], # универсальное ядро для нелинейных зависимостей
        'models__C': [0.1, 1, 10],
        'models__gamma': ['scale', 'auto', 0.1, 1],
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough'],

    }
]

In [ ]:
# инициализация класс для поиска гиперпараметров
grid_search = GridSearchCV(
  pipe_final,
  param_grid,
  scoring='roc_auc',
  n_jobs=-1,
  cv=5
) 

In [ ]:
# поиск лучших гиперпараметров
grid_search.fit(X_train, y_train)

In [ ]:
print ('Результат выбранной метрики на кросс-валидации:', round(grid_search.best_score_, 4))

In [ ]:
print('Лучшая модель и её параметры:\n\n', grid_search.best_params_)

In [ ]:
# предсказание целевого признака и определение вероятности для расчета метрик
y_test_pred = grid_search.predict(X_test)
y_test_proba = grid_search.predict_proba(X_test)

In [ ]:
# вывод метрики ROC_AUC для лучшей модели на тестовой выборке
print(f'Метрика ROC-AUC на тестовой выборке: {round(roc_auc_score(y_test, y_test_proba[:,1]), 4)}')
# вывод значения метрики F1 лучшей модели на тестовой выборке
print(f'Метрика F1-score на тестовой выборке: {round(f1_score(y_test, y_test_pred, average="binary"), 4)}')

__Промежуточный вывод по результату использования пайплайнов__:

В качестве метрики для оценки качества модели в прогнозировании целевога признака была выбрана __ROC_AUC__ из-за ее устойчивости к несбалансированности классов, а также ввиду ее удобства в использовании при сравнении моделей - чем выше метрика, тем лучше модель справляется с разделением классов. Кроме того __ROC-AUC__ оценивает классификатор по всем возможным порогам вероятности, в отличие от accuracy.
Лучшие результаты показала модель `SVC(kernel='rbf', C=1, degree=2, gamma=0.1, probability=True, random_state=42))`.
 - Значение метрики __ROC_AUC__ на тренировочной выборке - 0.9073;
 - Значение метрики __ROC_AUC__ на тестовой выборке - 0.9119;

В качестве уточняющей метрики для тестовой выборки была рассчитано значение __F1-score__ - 0.87, что говорит о балансе между точностью и полнотой модели.

Полученные метрики указывают на высокое качество выбранной модели.


## Анализ важности признаков

In [ ]:
# масштабирование и кодирование данных для использования в SHAP
X_1 = grid_search.best_estimator_['preprocessor'].fit_transform(X_train)


# получение названий столбцов
all_feature_names = grid_search.best_estimator_['preprocessor'].get_feature_names_out()

X_1 = pd.DataFrame(X_1, columns = all_feature_names)

explainer = shap.Explainer(grid_search.best_estimator_['models'].predict, X_1.values)
shap_values = explainer(X_1.sample(50)) # для экономии времени вычисления выбираем 50 произвольных строк

In [ ]:
# визуализация вклада признаков в каждое предсказание
shap.plots.beeswarm(shap_values, max_display=20)

In [ ]:
# отражение общего вклада признаков в прогнозы модели
shap.summary_plot(shap_values, plot_type='bar', max_display=20)

__Промежуточный вывод по анализу важности признаков__:

 - Наиболее важными признаками являются: `страниц_за_визит`, `минут_предыдущий_месяц`, `минут_текущий_месяц`, `выручка_препредыдущий_месяц`, `средний_просмотр_категорий_за_визит`, `неоплаченные продукты_штук_квартал`, `маркет_актив_6_мес`.
Стоит отметить, что модель посчитала, что значение выручки за один из прошлых периодов влияет на целевой показатель, хотя логически, это скорее совпадение.
 - Наименее важными являются признаки: `популярная_категория`, `разрешить_сообщать`, `тип_сервиса`.
 
 Резюмируя, наиболее важным является удержание внимания пользователей на сайте, а также проведение маркетинговых мероприятий, учитывая, что отсутствие запрета на рассылку на текущий момент не влияет на покупательскую активность. Не ясно предназначение типа сервиса, т.к. признак относится к группе с наименьшим влиянием. Возможно, стоит переработать эту систему.

## Сегментация покупателей

На стадии исследовательского анализа предполагалось, что склонность к покупкам товаров по акции влияет на покупательскую активность. Оценим зависимость прибыли от веростности снижения покупательской активности с учетом покупок по акции. Кроме того, проверим аналогичную зависимость с учетом других категориальных признаков.

In [ ]:
# дополнение тестовых данных вероятностью снижения активности и сведениями о прибыли
X_test_full = X_test.copy()
X_test_full['вероятность'] = y_test_proba[:, 1]
df_money = df_money.set_index('id')
X_test_full = X_test_full.join(df_money)

In [ ]:
# определение функции для отображения зависимости прибыли от вероятности снижения покупательской активности
# с учетом категориальных признаков
def display_scatterplots(df):
    sns.set(style='whitegrid', palette='muted', font_scale=1.2) 
    for column in df.columns:
        if df[column].dtype == 'object':
            fig = plt.figure(figsize=(12, 8))
            scatter = sns.scatterplot(
                data=df, 
                x='вероятность', 
                y='прибыль', 
                hue=column, 
                alpha=0.9,           
                sizes=(50, 200),    
                edgecolor="w",       
                linewidth=0.5        
            )
            scatter.legend(title=column, bbox_to_anchor=(1.05, 1), loc='upper left')
            plt.xlabel('Вероятность снижения активности', fontsize=14)
            plt.ylabel('Прибыль', fontsize=14)
            plt.title(f'Зависимость прибыли от вероятности снижения активности\n(с учетом \"{column}\")', fontsize=16)
            plt.tight_layout()
            plt.show()

In [ ]:
display_scatterplots(X_test_full)

По графикам видно, что прибыль не зависит от вероятности снижения покупательской активности. Распределение признаков `тип_сервиса`, `популярная_категория` и `разрешить_сообщать` равномерное, что говорит об отсутствии влияния на прибыль. При этом категория _много покупают по акции_ признака `акционные_покупки_объем` в большинстве своем сосредоточена в зоне высокой вероятности снижения покупательской активности и не влияет на величину прибыли.

Таким образом, в качестве исследуемого сегмента примем группа клиентов с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности.

In [ ]:
# объединение выборок для дальнейшего исследования
X_train_full = X_train.copy()
X_train_full['вероятность'] = grid_search.predict_proba(X_train)[:, 1]
X_train_full = X_train_full.join(df_money)
data = pd.concat([X_train_full, X_test_full])

Выделим пользователей с вероятностью снижения покупательской активности более 0.8 и со значением _много покупают по акции_ признака `акционные_покупки_объем`.

In [ ]:
data['исследуемый_сегмент'] = data.apply( lambda row: 'да' \
        if row['вероятность'] > 0.8 and row['акционные_покупки_объем'] == 'много покупают по акции' \
        else 'нет' \
      , axis=1)

Отображение распределений количественных для сегментированных и несегментированных пользователей

In [ ]:
# отображение распределений количественных признаков
for column in data.select_dtypes(include=['number']).columns:
    if column != 'вероятность':
        plt.figure(figsize=(15,8))
        plt.subplot(1, 2, 1)
        plt.title(f'Гистограмма для "{column}"')
        sns.histplot(data=data,
                     x=column,
                     hue='исследуемый_сегмент',
                     palette='YlGn',
                     kde=True)
        plt.xlabel(f'Признак "{column}"')
        plt.ylabel('Частота')
        plt.tight_layout()

In [ ]:
# построение круговой диаграммы по распределению категорий товаров для выбранного сегмента
data[data['исследуемый_сегмент'] == 'да']['популярная_категория'].value_counts().plot(
    kind='pie',
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'black'},
    figsize=(8, 8)
)
plt.title('Распределение категорий товаров для выбранного сегмента')
plt.ylabel('')

plt.show()

__Промежуточный вывод по анализу выбранного сегмента пользователей__:

В качестве исследуемого сегмента была выбрана группа клиентов с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности. 
При сопоставлении распределений выделенный сегмент в целом меньше тратит времени на посещение сайта и просмотр необходимых категорий (преимущественно не более 3х). 

Самыми популярными категорями у выбранного группы являются:
 - товары для детей 35.7%;
 - косметика и аксессуары 19.6%;
 - домашний текстиль 18.8% 
 
Суммарно на 3 категории приходится почти 75% внимания.
 
Учитывая, что на покупательскую активность в большей степени влияет время, проведенное на сайте, а также большое количество совершенных покупок по акции, то предлагается рассмотреть возможность комбинирования акционных предложений - стратегия «купи комплект» (например, скидки при покупке нескольких товаров из разных категорий). Введение бонусных баллов за время, проведенное на сайте, которые могут быть обменены на скидки или дополнительные бонусы, также может поспособствовать увеличению покупательской активности.

Ввиду того, что выбранные покупатели просматривают преимущественно не более 5 страниц сайта, предлагается разместить на первых информационный контент (статьи, видео) и интерактивные элементы (квизы или тесты), опираясь на предпочтения, с призами, которые в дальнейшем могут быть использованы в интернет-магазине.

Учитывая наличие большего количества неоплаченных товаров в корзине у выбранной группы, рекомендуется активация временных акций (например, ограниченные по времени скидки), для побуждения клиентов завершить/совершить покупки.

Дополнительно предлагается доработать премимум сервис, т.к., судя по имеющимся данным, он мало на что влияет.
 

<div class="alert alert-success">
<b>ОТЛИЧНО! 👍</b>

Молодец, в разделе выбора сегмента и разработки маркетинговых предложений по увеличению покупательской активности для него я не вижу никаких проблем. Проведен и аналитический и графический анализ, в выводе находится саммари по всем, наиболее значимым поинтам анализа. Что же касается рекомендаций, то конечно здесь написать можно много ... твой анализ (и графики для признаков) дают массу полезной информации о характеристиках 2х групп покупателей. Написать можно много, но для начала достаточно и тех моментов, которые упоминаешь здесь ты. Молодец!
</div>

## Общий вывод

__Задачи проекта:__

- подготовить модель, которая предскажет вероятность снижения покупательской активности клиента в интрнет магазине "В один клик";
- используя данные, подготовленной модели, выделить сегмент покупателей и разработать для них персонализированные предложания.

__Исходные данные:__

- Таблица, которая содержит данные о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении.
- Таблица с данными о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.
- Таблица с данными о времени (в минутах), которое покупатель провёл на сайте в течение периода.
- Таблица с данными о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.

Все таблицы были представлены в формате .csv. 

__Предобработка данных:__

Исходные данные были проверены на наличие пропусков и дубликатов. Устранены неявные дубликаты в названиях значений некоторых признаков, а также устранены опечатки. 

По результату проведения исследовательского и корреляционного анализов был преобразован количественный признак `акционные_покупки` в бинарный категориальные (много покупают по акции и мало покупают по акции), а также объединены признаки `выручка_текущий_месяц` и `выручка_предыдущий_месяц` в один - `выручка_последние_2_месяца`.

__Выбор модели:__

При поиске лучшей модели учитывались:
- объем данных;
- результаты исследовательского и корреляционного анализов.

Было выполнено обучение 4х видов моделей со сплошным перебором нескольких гиперпараметров.

В качестве метрики для оценки качества модели в прогнозировании целевога признака (покупательской активности) была выбрана __ROC_AUC__.

__ROC-AUC__  оценивает качество модели путем измерения площади под кривой ROC (Receiver Operating Characteristic) – графика, который показывает зависимость между долей правильных предсказаний (True Positive Rate) и долей ошибочных предсказаний (False Positive Rate)

Выбор сделан из-за ее устойчивости к несбалансированности классов, а также ввиду ее удобства в использовании при сравнении моделей - чем выше метрика, тем лучше модель справляется с разделением классов. 
Лучшие результаты показала модель `SVC(kernel='rbf', C=1, degree=2, gamma=0.1, probability=True, random_state=42))`.
 - Значение метрики __ROC_AUC__ на тренировочной выборке - 0.9073;
 - Значение метрики __ROC_AUC__ на тестовой выборке - 0.9119;

В качестве уточняющей метрики для тестовой выборки было рассчитано значение __F1-score__ - 0.87, что говорит о балансе между точностью и полнотой модели.

Полученные метрики указывают на высокое качество выбранной модели.

Также выполнена оценка важности признаков:

 - Наиболее важными признаками являются: `страниц_за_визит`, `минут_предыдущий_месяц`, `минут_текущий_месяц`, `средний_просмотр_категорий_за_визит`, `неоплаченные продукты_штук_квартал`, `маркет_актив_6_мес`, иными словами, речь идет об удержании внимания пользователей.

 - Наименее важными являются признаки: `популярная_категория`, `разрешить_сообщать`, `тип_сервиса`.

__Рекомендации для выбранного сегмента пользвателей__:

В качестве исследуемого сегмента была выбрана группа клиентов с максимальной долей покупок по акции и высокой вероятностью снижения покупательской активности. 

Рекомендации:
- рассмотреть возможность комбинирования акционных предложений - стратегия «купи комплект» (например, скидки при покупке нескольких товаров из разных категорий). У выбранного сегмента наиболее популярными являются 3 категории, на долю которых приходится почти 75% покупок:  - товары для детей 35.7%; косметика и аксессуары 19.6%; домашний текстиль 18.8%.
- рассмотреть возможность размещение на первых страницах акционных предложений, полезной информации и иных, цепляющих внимание, сущеностей. Пользователи предпочитают просматривать не более 5 страниц.
- рассмотреть возможность геймификации акционных и бонусных предложений. Задача заинтересовать пользователей, которые предпочитают выгоду.
- рекомендуется активация временных акций (например, ограниченные по времени скидки), для побуждения клиентов завершить/совершить покупки. У выбранных пользователей среднее количество неоплаченных товаров ваше чем в среднем по выборке. 